In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from splinter import Browser

In [2]:
# Send a GET request to the website
url = "https://aqicn.org/city/all/"

response = requests.get(url)

In [3]:
# Create a BeautifulSoup object to parse the HTML content
soup = BeautifulSoup(response.content, "html.parser")

In [4]:
container = soup.find("div", class_="main-cities")

In [5]:
city_elements = container.find_all("a")

In [6]:
cities = [city_element.text.strip() for city_element in city_elements]

In [7]:
cities_df = pd.DataFrame(cities, columns=["City Name"])
cities_df

,City Name
0,Shanghai (上海)
1,Beijing (北京)
2,Tianjin (天津)
3,Guangzhou (广州市)
4,Shenzhen (深圳)
...,...
528,Addis Ababa (አዲስ አበባ)
529,Nairobi
530,Kampala (كامبالا)
531,Algiers (الجزائر)


In [8]:
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   City Name  533 non-null    object
dtypes: object(1)
memory usage: 4.3+ KB


In [10]:
#Removing parenthesis form the City Name column
cities_df['City Name'] = cities_df['City Name'].str.replace(r'\s+\(.*\)', '', regex=True)
cities_df

,City Name
0,Shanghai
1,Beijing
2,Tianjin
3,Guangzhou
4,Shenzhen
...,...
528,Addis Ababa
529,Nairobi
530,Kampala
531,Algiers


In [11]:
cities_list = cities_df["City Name"].tolist()

In [12]:
print(cities_list)

['Shanghai', 'Beijing', 'Tianjin', 'Guangzhou', 'Shenzhen', 'Wuhan', 'Dongguan', 'Chongqing', 'Chengdu', 'Nanjing', 'Taipei', 'Kaohsiung', 'Taichung', 'tainan', 'banqiao', 'hsinchu', 'taoyuan-city', 'Keelung', 'Hong Kong', 'Macao', 'Hanyang', 'Busan', 'Incheon', 'Daejeon', 'Ulsan', 'Daegu', 'Gwangju', 'Suwon', 'Goyang', 'Seongnam', 'Edo', 'Yokohama', 'Osaka', 'Nagoya', 'Sapporo', 'Kobe', 'Kyoto', 'Fukuoka', 'Kawasaki', 'saitama', 'Moscow', 'Krasnoyarsk', 'Kaliningrad', 'Leningrad', 'Novo-Nikolaevsk', 'Nizhniy Novgorod', 'Chelyabinsk', 'Ufa', 'Dhaka', 'Kathmandu', 'Pokhara', 'Patan', 'biratnagar', 'Birgunj', 'dharan-bazar', 'Bharatpur', 'Bombay', 'Delhi', 'Bangalore', 'Calcutta', 'Chennai', 'Ahmedabad', 'Hyderabad', 'Pune', 'Kanpur', 'Bangkok', 'mueang-samut-prakan', 'Nonthaburi', 'chon-buri', 'Nakhon Ratchasima', 'Chiangmai', 'Hat Yai', 'Pak Kret', 'si-racha', 'Amphoe Phra Pradaeng', 'Lampang', 'surin', 'Vientiane', 'Rangoon', 'Kota Bharu', 'Kuala Lumpur', 'klang', 'kampung-baru-subang

In [13]:
import json
import time

In [14]:
base_url_cities_0 = "http://api.waqi.info/feed/"
base_url_cities_1 = "/?token=cfec51e63370e90b7880950705e8add14a2602b6"
base_url_citydata_0 = "http://api.airvisual.com/v2/city?city="

for x in cities_list:
    base_url_cities = base_url_cities_0 + x + base_url_cities_1
    params = {
        "param":"pm25",
        "value":"true"
        
    }
    response_cities = requests.get(base_url_cities, params=params).json()
    print(json.dumps(response_cities, indent=4))

{
    "status": "ok",
    "data": {
        "aqi": 93,
        "idx": 1437,
        "attributions": [
            {
                "url": "https://sthj.sh.gov.cn/",
                "name": "Shanghai Environment Monitoring Center(\u4e0a\u6d77\u5e02\u73af\u5883\u76d1\u6d4b\u4e2d\u5fc3)"
            },
            {
                "url": "http://106.37.208.233:20035/emcpublish/",
                "name": "China National Urban air quality real-time publishing platform (\u5168\u56fd\u57ce\u5e02\u7a7a\u6c14\u8d28\u91cf\u5b9e\u65f6\u53d1\u5e03\u5e73\u53f0)"
            },
            {
                "url": "https://china.usembassy-china.org.cn/embassy-consulates/shanghai/air-quality-monitor-stateair/",
                "name": "U.S. Consulate Shanghai Air Quality Monitor"
            },
            {
                "url": "https://waqi.info/",
                "name": "World Air Quality Index Project"
            }
        ],
        "city": {
            "geo": [
                31.2047372

{
    "status": "ok",
    "data": {
        "aqi": 82,
        "idx": 1452,
        "attributions": [
            {
                "url": "http://113.108.142.147:20035/emcpublish/",
                "name": "China National Urban air quality real-time publishing platform (\u5168\u56fd\u57ce\u5e02\u7a7a\u6c14\u8d28\u91cf\u5b9e\u65f6\u53d1\u5e03\u5e73\u53f0)"
            },
            {
                "url": "http://www.tjemc.org.cn/",
                "name": "Tianjin Environmental Monitoring Center (\u5929\u6d25\u5e02\u73af\u5883\u76d1\u6d4b\u4e2d\u5fc3)"
            },
            {
                "url": "https://waqi.info/",
                "name": "World Air Quality Index Project"
            }
        ],
        "city": {
            "geo": [
                39.084158,
                117.200983
            ],
            "name": "Tianjin (\u5929\u6d25)",
            "url": "https://aqicn.org/city/tianjin",
            "location": ""
        },
        "dominentpol": "pm25",
     

{
    "status": "ok",
    "data": {
        "aqi": 109,
        "idx": 1539,
        "attributions": [
            {
                "url": "http://meeb.sz.gov.cn/",
                "name": "Shenzhen Environment Network (\u6df1\u5733\u4eba\u5c45\u73af\u5883\u7f51)"
            },
            {
                "url": "http://gdee.gd.gov.cn/",
                "name": "Guangdong Environmental Protection public network (\u5e7f\u4e1c\u73af\u5883\u4fdd\u62a4\u516c\u4f17\u7f51)"
            },
            {
                "url": "http://www.szhec.gov.cn/",
                "name": "Shenzhen Environment Network (\u6df1\u5733\u4eba\u5c45\u73af\u5883\u7f51)"
            },
            {
                "url": "https://waqi.info/",
                "name": "World Air Quality Index Project"
            }
        ],
        "city": {
            "geo": [
                22.543099,
                114.057868
            ],
            "name": "Shenzhen (\u6df1\u5733)",
            "url": "https://aqi

{
    "status": "ok",
    "data": {
        "aqi": 99,
        "idx": 1547,
        "attributions": [
            {
                "url": "http://www.gdep.gov.cn/",
                "name": "Guangdong Environmental Protection public network (\u5e7f\u4e1c\u73af\u5883\u4fdd\u62a4\u516c\u4f17\u7f51)"
            },
            {
                "url": "http://gdee.gd.gov.cn/",
                "name": "Guangdong Environmental Protection public network (\u5e7f\u4e1c\u73af\u5883\u4fdd\u62a4\u516c\u4f17\u7f51)"
            },
            {
                "url": "https://waqi.info/",
                "name": "World Air Quality Index Project"
            }
        ],
        "city": {
            "geo": [
                23.020536,
                113.751765
            ],
            "name": "Dongguan (\u4e1c\u839e)",
            "url": "https://aqicn.org/city/dongguan",
            "location": ""
        },
        "dominentpol": "pm25",
        "iaqi": {
            "co": {
                "

{
    "status": "ok",
    "data": {
        "aqi": 114,
        "idx": 1450,
        "attributions": [
            {
                "url": "https://sthjt.sc.gov.cn/",
                "name": "Sichuan Province Environmental Protection Agency (\u56db\u5ddd\u7701\u73af\u4fdd\u91cd\u70b9\u57ce\u5e02\u73af\u5883\u7a7a\u6c14\u8d28\u91cf\u5b9e\u65f6\u76d1\u6d4b\u7ed3\u679c)"
            },
            {
                "url": "http://sthj.chengdu.gov.cn/",
                "name": "Chengdu Environmental Protection Agency (\u6210\u90fd\u5e02\u73af\u5883\u76d1\u6d4b\u4e2d\u5fc3\u7ad9_\u6210\u90fd\u5e02\u73af\u5883\u76d1\u6d4b\u4e2d\u5fc3\u7ad9)"
            },
            {
                "url": "http://106.37.208.233:20035/emcpublish/",
                "name": "China National Urban air quality real-time publishing platform (\u5168\u56fd\u57ce\u5e02\u7a7a\u6c14\u8d28\u91cf\u5b9e\u65f6\u53d1\u5e03\u5e73\u53f0)"
            },
            {
                "url": "https://china.usembassy-china.o

...

{
    "status": "ok",
    "data": {
        "aqi": 24,
        "idx": 10833,
        "attributions": [
            {
                "url": "https://luft.hamburg.de/",
                "name": "Hamburger air monitoring network - (Hamburger Luftmessnetz \u2013 FHH)",
                "logo": "Germany-Hamburg.png"
            },
            {
                "url": "https://waqi.info/",
                "name": "World Air Quality Index Project"
            }
        ],
        "city": {
            "geo": [
                53.59235,
                10.053724
            ],
            "name": "Habichtstra\u00dfe, Hamburg, Germany",
            "url": "https://aqicn.org/city/germany/hamburg/habichtstrasse",
            "location": ""
        },
        "dominentpol": "o3",
        "iaqi": {
            "co": {
                "v": 0.1
            },
            "dew": {
                "v": 7
            },
            "h": {
                "v": 93
            },
            "no2": {
      

{
    "status": "ok",
    "data": {
        "aqi": 46,
        "idx": 6099,
        "attributions": [
            {
                "url": "http://www.eea.europa.eu/themes/air/",
                "name": "European Environment Agency",
                "logo": "Europe-EEA.png"
            },
            {
                "url": "https://www.lanuv.nrw.de/luft/luft.htm",
                "name": "LANUV - State Agency for Nature, Environment and Consumer Protection of North Rhine Westphalia (Landesamt f\u00fcr Natur, Umwelt und Verbraucherschutz NRW)",
                "logo": "de-lanuv.png"
            },
            {
                "url": "https://waqi.info/",
                "name": "World Air Quality Index Project"
            }
        ],
        "city": {
            "geo": [
                50.947268,
                6.957649
            ],
            "name": "K\u00f6ln Turiner Stra\u00dfe, Germany",
            "url": "https://aqicn.org/city/germany/nrw/koln-turiner-strasse",
      

{
    "status": "ok",
    "data": {
        "aqi": 21,
        "idx": 6089,
        "attributions": [
            {
                "url": "https://www.lanuv.nrw.de/luft/luft.htm",
                "name": "LANUV - State Agency for Nature, Environment and Consumer Protection of North Rhine Westphalia (Landesamt f\u00fcr Natur, Umwelt und Verbraucherschutz NRW)",
                "logo": "de-lanuv.png"
            },
            {
                "url": "http://www.eea.europa.eu/themes/air/",
                "name": "European Environment Agency",
                "logo": "Europe-EEA.png"
            },
            {
                "url": "https://waqi.info/",
                "name": "World Air Quality Index Project"
            }
        ],
        "city": {
            "geo": [
                51.451176,
                7.030534
            ],
            "name": "Essen-Ost Steeler Stra\u00dfe, Germany",
            "url": "https://aqicn.org/city/germany/nrw/essen-ost-steeler-strasse",
 

{
    "status": "ok",
    "data": {
        "aqi": 51,
        "idx": 6078,
        "attributions": [
            {
                "url": "https://www.lanuv.nrw.de/luft/luft.htm",
                "name": "LANUV - State Agency for Nature, Environment and Consumer Protection of North Rhine Westphalia (Landesamt f\u00fcr Natur, Umwelt und Verbraucherschutz NRW)",
                "logo": "de-lanuv.png"
            },
            {
                "url": "http://www.eea.europa.eu/themes/air/",
                "name": "European Environment Agency",
                "logo": "Europe-EEA.png"
            },
            {
                "url": "https://waqi.info/",
                "name": "World Air Quality Index Project"
            }
        ],
        "city": {
            "geo": [
                51.523543,
                7.483562
            ],
            "name": "Dortmund Brackeler Stra\u00dfe, Germany",
            "url": "https://aqicn.org/city/germany/nrw/dortmund-brackeler-strasse",

KeyboardInterrupt: 

In [17]:
#Storage the data into a MongoDB database
from pymongo import MongoClient

In [18]:
#Establish the MongoDB server
client = MongoClient('mongodb://localhost:27017/')

In [19]:
#Create a database
db = client['major_cities_airq']

In [22]:
# Create a collection
collection = db['major_cities_airq']

In [23]:
#Prepare the data to be inserted into the collection
airquiality = {
    'aqi': response_cities['data']['aqi'],
    'idx': response_cities['data']['idx'],
    'attributions': response_cities['data']['attributions'],
    'city_geo': response_cities['data']['city']['geo'],
    'city_name': response_cities['data']['city']['name'],
    'dominant_pol': response_cities['data']['dominentpol'],
    'iaqi': response_cities['data']['iaqi'],
    'time': response_cities['data']['time'],
    'forecast': response_cities['data']['forecast']
    
}

In [24]:
#Insert the data into the collection
collection.insert_one(airquiality)

In [25]:
client.close()